In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
#@save
def masked_softmax(X, valid_lens):
    """通过在最后一个轴上掩蔽元素来执行softmax操作"""
    # X:3D张量，valid_lens:1D或2D张量
    if valid_lens is None:
        return nn.functional.softmax(X, dim=-1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
        else:
            valid_lens = valid_lens.reshape(-1)
        # 最后一轴上被掩蔽的元素使用一个非常大的负值替换，从而其softmax输出为0
        X = d2l.sequence_mask(X.reshape(-1, shape[-1]), valid_lens,
                              value=-1e6)
        return nn.functional.softmax(X.reshape(shape), dim=-1)

In [3]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([2, 3]))

tensor([[[0.4211, 0.5789, 0.0000, 0.0000],
         [0.6871, 0.3129, 0.0000, 0.0000]],

        [[0.5045, 0.2563, 0.2392, 0.0000],
         [0.2209, 0.3908, 0.3882, 0.0000]]])

In [4]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([[1, 3], [2, 4]]))

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.3714, 0.3246, 0.3040, 0.0000]],

        [[0.5477, 0.4523, 0.0000, 0.0000],
         [0.1674, 0.3794, 0.1717, 0.2815]]])